##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [2]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
import numpy as np

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.42671522,  0.05397483,  0.05025768, -0.23283583,  0.19140692,
         0.20172037, -0.5711364 , -0.24349491, -0.20750986,  0.4980408 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.1435624 , 0.09889212, 0.0985252 , 0.07423375, 0.11346127,
        0.1146375 , 0.05292725, 0.07344668, 0.0761378 , 0.1541761 ]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [9]:
loss_fn(y_train[:1], predictions).numpy()

2.1659803

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2938 - accuracy: 0.9143
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1424 - accuracy: 0.9585
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1077 - accuracy: 0.9678
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0883 - accuracy: 0.9726
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0738 - accuracy: 0.9759


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [12]:
res = model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0786 - accuracy: 0.9761


In [13]:
predictions = model(x_test)
predictions = tf.nn.softmax(predictions)
predictions = tf.math.argmax(predictions, axis=1).numpy()
comp1 = np.equal(predictions, y_test)

orig = 0
for i in comp1:
    if i:
        orig+=1
print(f"Number of samples classified correctly = {orig}")  

Number of samples classified correctly = 9761


The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [14]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [15]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.8071498e-11, 4.0838489e-11, 1.5699646e-07, 9.1320571e-06,
        3.0077254e-13, 2.9943905e-08, 1.2085041e-14, 9.9999058e-01,
        2.9685316e-09, 6.6335446e-08],
       [1.1365912e-09, 4.2176498e-06, 9.9998248e-01, 1.3352707e-05,
        1.8604250e-16, 9.3441432e-09, 3.1143590e-09, 8.9961277e-15,
        6.9544048e-10, 1.3392569e-17],
       [5.9652655e-07, 9.9883407e-01, 4.5308265e-05, 1.1473447e-05,
        6.2388230e-05, 7.3864567e-07, 1.1485215e-05, 1.0096903e-03,
        2.4059595e-05, 1.9997054e-07],
       [9.9990547e-01, 4.0933212e-10, 5.6634748e-05, 2.3605881e-07,
        7.8420662e-08, 1.7211945e-05, 4.0442906e-06, 1.2796212e-05,
        2.3173897e-08, 3.4197787e-06],
       [3.0533668e-06, 8.3445623e-10, 2.0260713e-05, 1.4391137e-08,
        9.9568224e-01, 1.2827153e-06, 6.4109190e-07, 1.4776895e-04,
        4.8512942e-07, 4.1442015e-03]], dtype=float32)>

•	FGSM based untargeted attacks

In [16]:
x_tensor = tf.convert_to_tensor(x_test, dtype=tf.float32)
with tf.GradientTape() as tape:
    tape.watch(x_tensor)
    output = model(x_tensor)
    loss_grad = loss_fn(y_test, output)

gradients = tape.gradient(loss_grad, x_tensor)
gradients_sign = tf.sign(gradients) 

epsilons = [1/255, 5/255, 10/255, 20/255, 30/255, 40/255, 50/255]
x_primes_u = []
for epsilon in epsilons:
    x_primes_u.append(tf.clip_by_value((x_tensor + epsilon*gradients_sign), clip_value_min=0.0, clip_value_max=1.0))  

y_preds = []
for i in range(len(epsilons)):
    predictions = model(x_primes_u[i]).numpy()
    probs = tf.nn.softmax(predictions)
    y_preds.append(tf.math.argmax(probs, axis=1).numpy()) 

In [17]:
for p in range(7):
    count = 0
    for i in range(len(y_test)):
        if comp1[i]==True:
            if y_test[i]!=y_preds[p][i]:
                count+=1
    print(f"Epsilon = {epsilons[p]*255}, success rate of attack = {round((count/orig)*100,2)}%" )

Epsilon = 1.0, success rate of attack = 0.64%
Epsilon = 5.0, success rate of attack = 4.94%
Epsilon = 10.0, success rate of attack = 17.94%
Epsilon = 20.0, success rate of attack = 62.03%
Epsilon = 30.0, success rate of attack = 84.84%
Epsilon = 40.0, success rate of attack = 94.22%
Epsilon = 50.0, success rate of attack = 97.53%


• FGSM based targeted attacks

In [18]:
y_test_mod = []
for i in y_test:
    y_test_mod.append((i+1)%10)

x_tensor = tf.convert_to_tensor(x_test, dtype=tf.float32)
with tf.GradientTape() as tape:
    tape.watch(x_tensor)
    output = model(x_tensor)
    loss_grad = loss_fn(y_test_mod, output)

gradients = tape.gradient(loss_grad, x_tensor)
gradients_sign = tf.sign(gradients) 

epsilons = [1/255, 5/255, 10/255, 20/255, 30/255, 40/255, 50/255]
x_primes_t = []
for epsilon in epsilons:
    x_primes_t.append(tf.clip_by_value((x_tensor - epsilon*gradients_sign), clip_value_min=0.0, clip_value_max=1.0))  

y_preds = []
for i in range(len(epsilons)):
    predictions = model(x_primes_t[i])
    probs = tf.nn.softmax(predictions)
    y_preds.append(tf.math.argmax(probs, axis=1).numpy())  

In [19]:
for p in range(7):
    count = 0
    for i in range(len(y_test)):
        if comp1[i]==True:
            if y_test[i]+1==y_preds[p][i]:
                count+=1
    print(f"Epsilon = {epsilons[p]*255}, success rate of attack = {round((count/orig)*100,2)}%" )

Epsilon = 1.0, success rate of attack = 0.02%
Epsilon = 5.0, success rate of attack = 0.41%
Epsilon = 10.0, success rate of attack = 2.22%
Epsilon = 20.0, success rate of attack = 21.69%
Epsilon = 30.0, success rate of attack = 35.89%
Epsilon = 40.0, success rate of attack = 47.81%
Epsilon = 50.0, success rate of attack = 54.38%


•	Adversarial Retraining against Untargeted FGSM Attacks

In [20]:
x_tensor = tf.convert_to_tensor(x_train, dtype=tf.float32)
with tf.GradientTape() as tape:
    tape.watch(x_tensor)
    output = model(x_tensor)
    loss_grad = loss_fn(y_train, output)

gradients = tape.gradient(loss_grad, x_tensor)
gradients_sign = tf.sign(gradients) 

epsilons = [10/255]
x_primes = []
for epsilon in epsilons:
    x_primes.append(tf.clip_by_value((x_tensor + epsilon*gradients_sign), clip_value_min=0.0, clip_value_max=1.0))  

Combining original training data and perturbed training data

In [21]:
x_train_mod = np.concatenate((x_train, x_primes[0]), axis=0)
y_train_mod = np.concatenate((y_train, y_train), axis=0)

Retraining the baseline DNN with a new training dataset containing both images from the original training dataset and the new adversarially perturbed images.

In [22]:
model2 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

Training a new model for the new training data

In [23]:
model2.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model2.fit(x_train_mod, y_train_mod, epochs=5)

Epoch 1/5
3750/3750 [==============================] - 6s 2ms/step - loss: 0.2002 - accuracy: 0.9421
Epoch 2/5
3750/3750 [==============================] - 6s 2ms/step - loss: 0.0787 - accuracy: 0.9757
Epoch 3/5
3750/3750 [==============================] - 5s 1ms/step - loss: 0.0567 - accuracy: 0.9823
Epoch 4/5
3750/3750 [==============================] - 6s 1ms/step - loss: 0.0462 - accuracy: 0.9850
Epoch 5/5
3750/3750 [==============================] - 6s 1ms/step - loss: 0.0407 - accuracy: 0.9866


Classification accuracy of the adversarially retrained DNN on the original test dataset that contains only clean inputs.

In [24]:
model2.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0839 - accuracy: 0.9760


[0.08385971933603287, 0.9760000109672546]

In [25]:
predictions = model2(x_test)
predictions = tf.nn.softmax(predictions)
predictions = tf.math.argmax(predictions, axis=1).numpy()
comp1 = np.equal(predictions, y_test)

orig = 0
for i in comp1:
    if i:
        orig+=1
print(f"Number of samples classified correctly = {orig}")  

Number of samples classified correctly = 9760


FGSM based untargeted attacks using images from the clean test set on the adversarially retrained DNN. 

In [31]:
x_tensor = tf.convert_to_tensor(x_test, dtype=tf.float32)
with tf.GradientTape() as tape:
    tape.watch(x_tensor)
    output = model2(x_tensor)
    loss_grad = loss_fn(y_test, output)

gradients = tape.gradient(loss_grad, x_tensor)
gradients_sign = tf.sign(gradients) 

epsilons = [10/255]
x_primes = []
for epsilon in epsilons:
    x_primes.append(tf.clip_by_value((x_tensor + epsilon*gradients_sign), clip_value_min=0.0, clip_value_max=1.0))  

y_preds = []
for i in range(len(epsilons)):
    predictions = model2(x_primes[i]).numpy()
    probs = tf.nn.softmax(predictions)
    y_preds.append(tf.math.argmax(probs, axis=1).numpy())  

In [32]:
for p in range(len(epsilons)):
    count = 0
    for i in range(len(y_test)):
        if comp1[i]==True:
            if y_test[i]!=y_preds[p][i]:
                count+=1
    print(f"Epsilon = {epsilons[p]*255}, success rate of attack = {round((count/orig)*100,2)}%" )

Epsilon = 10.0, success rate of attack = 79.9%


•	Is the adversarially retrained DNN robust against adversarial perturbations? 
Answer: No, the adversarially retrained DNN was still susceptible to untargetted FSGM attacks.